# Fetch Forecasts
Welcome to this tutorial! In this notebook, we will see how to retrieve forecasts from ensemble models, such as PEAROME.
In these models, a set (or ensemble) of forecasts is produced. Each forecast is achieved with the same numeric model (AROME for PEAROME), but with slightly different initialization parameters. This set of forecasts aims to give an indication of the range of possible weather conditions. Thus it gives a probabilistic approach of the weather forecast.
PEAROME produces a set of 25 members, so 25 possible scenarios for each of its available indicator.


We will cover the following cases:
- Retrieving data of a determinist model and a ensemble model
- For a single run date
- Specifying the "interval" parameter for each indicator
- Specifying the "forecast_horizon" parameter
- Specifying the "heights" or "pressures" parameters for each indicator



In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from meteole import AromeForecast, AromePEForecast  # or ArpegeForecast, PiafForecast, AromePIForecast

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("meteole")

**Requirements notice** : TODO Link to the documentation to have application_id

In [ ]:
APP_ID = "TO BE SET"

## Init Client

In [ ]:
# init client
arome_client = AromeForecast(application_id=APP_ID)
pearome_client = AromePEForecast(application_id=APP_ID)

In [ ]:
# Available indicators for AROME
arome_client.indicators

In [ ]:
# Available indicators for PEAROME
pearome_client.indicators

In [ ]:
# pick a indicator
indicator = "TEMPERATURE__SPECIFIC_HEIGHT_LEVEL_ABOVE_GROUND"

### Get parameters

If you want to select the different parameters available, you can retrieve them using the `get_capabilities` and `get_coverage_description`.


In [ ]:
# get all the capabilities of the model
aro_capabilities = arome_client.get_capabilities()

# select one run, the most recent for example
most_recent_run_aro = pd.to_datetime(aro_capabilities["run"].max(), format="%Y-%m-%dT%H.%M.%SZ")

# get the coverage description for this indicator and this run
aro_cov_descr = arome_client.get_coverage_description(
    indicator + "___" + most_recent_run_aro.strftime("%Y-%m-%dT%H.%M.%SZ")
)

print(aro_cov_descr)

### Fetch data using params of your choice

In [ ]:
# Get for the AROME model, the forecast of Temperature at 2 meters for 24 first available forecast horizons
df_arome = arome_client.get_coverage(
    indicator=indicator,
    interval=None,
    forecast_horizons=aro_cov_descr["forecast_horizons"],
    heights=[2],
    pressures=None,
    long=(1.0, 1.5),
    lat=(43, 43.5),
    temp_dir=None,
)
print(df_arome.head())

In [ ]:
# It is also possible to specify one particular location instead of a region, or a combination of both (for example a single lat value and a range for long)
df_arome = arome_client.get_coverage(
    indicator=indicator,
    interval=None,
    forecast_horizons=aro_cov_descr["forecast_horizons"],
    heights=[2],
    pressures=None,
    long=1.25,
    lat=43.25,
    temp_dir=None,
)
print(df_arome.head())

In [ ]:
# Same for PEAROME
# get all the capabilities of the model
pearo_capabilities = pearome_client.get_capabilities()

# select one run, the most recent for example
most_recent_run_pearo = pd.to_datetime(pearo_capabilities["run"].max(), format="%Y-%m-%dT%H.%M.%SZ")

# get the coverage description for this indicator and this run
pearo_cov_descr = pearome_client.get_coverage_description(
    indicator + "___" + most_recent_run_pearo.strftime("%Y-%m-%dT%H.%M.%SZ")
)

print(pearo_cov_descr)

In [ ]:
# For the PEArome, get_coverage_description retrieve the description available for the member 0 by default.
# However, it is possible to specify several members to compare the description of each member.
# It can be useful when the forecast horizons are not all the same accross members
# (it can happen when the model is being updated.)
pearo_cov_descr_mult = pearome_client.get_coverage_description(
    indicator + "___" + most_recent_run_pearo.strftime("%Y-%m-%dT%H.%M.%SZ"), ensemble_numbers=[0, 1, 2]
)

print(pearo_cov_descr_mult)

In [ ]:
# Get for the PEAROME model, the forecast of Temperature at 2 meters for 24 first available forecast horizons
# For Ensemble models, the members (the numbers of each set of forecast) to retrieve must be specified
# with the parameter ensemble_numbers (in this example, we retrieve the 5 first members).
# PEAROME provides 25 members, from 0 to 24. Note that member 0 is the same as the AROME determinist model.
df_pearome = pearome_client.get_coverage(
    indicator=indicator,
    interval=None,
    forecast_horizons=pearo_cov_descr["forecast_horizons"],
    ensemble_numbers=range(5),
    heights=[2],
    pressures=None,
    long=(1.0, 1.5),
    lat=(43, 43.5),
    temp_dir=None,
)

# Note that the resulting dataframe has a new column "number" identifying each member of the ensemble forecast.
print(df_pearome.head())

In [ ]:
# Illustration by plotting each member of the PEAROME model (thin lines),
# along with the single forecast of AROME model (dotted line)
# for a given location.
lat = 43
lon = 1

df_arome_single_loc = df_arome.loc[
    (df_arome["latitude"].round(2) == lat) & (df_arome["longitude"].round(2) == lon)
].copy()
df_arome_single_loc.index = df_arome_single_loc["run"] + df_arome_single_loc["forecast_horizon"]
df_pearome_single_loc = df_pearome.loc[
    (df_pearome["latitude"].round(2) == lat) & (df_pearome["longitude"].round(2) == lon)
].copy()
df_pearome_single_loc["validity_time"] = df_pearome_single_loc["run"] + df_pearome_single_loc["forecast_horizon"]

# reshape the result to have one column per member
reshaped_df = df_pearome_single_loc.pivot(index="validity_time", columns="ensemble_number", values="t_2m")
reshaped_df.columns = [f"PEAro_n_{int(col)}" for col in reshaped_df.columns]

reshaped_df.plot(linewidth=0.5)
df_arome_single_loc["t_2m"].plot(linewidth=2, linestyle="--", label="Arome")
plt.legend()